In [1]:
#from bracket_builder import bracketeer
# directory structure should be /.../datapath/files.csv
datapath = "data/"
submissionPath = "submit.csv"
outputFilePath = 'output.png'
#b = bracketeer(2017, datapath)
#b.make_bracket(submissionPath, outputFilePath)

In [2]:
#import binarytree as bt
from binarytree import Node
import os
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

In [3]:
class extNode(Node):
    def __init__(self, value, left=None, right=None, parent=None):
        Node.__init__(self, value, left=left, right=right)
        if parent is not None and isinstance(parent, extNode):
             self.__setattr__('parent', parent)
        else:
             self.__setattr__('parent', None)
    def __setattr__(self, name, value):
        # Magically set the parent to self when a child is created
        if (name in ['left', 'right']
                and value is not None
                and isinstance(value, extNode)):
            value.parent = self
        object.__setattr__(self, name, value)


def build(values):
    nodes = [None if v is None else extNode(v) for v in values]

    for index in range(1, len(nodes)):
        node = nodes[index]
        if node is not None:
            parent_index = (index - 1) // 2
            parent = nodes[parent_index]
            if parent is None:
                raise NodeNotFoundError(
                    'Parent node missing at index {}'
                    .format(parent_index)
                )
            setattr(parent, 'left' if index % 2 else 'right', node)

    return nodes[0] if nodes else None


In [4]:
teams_df = pd.read_csv(os.path.join(datapath, "Teams.csv"))
seeds_df = pd.read_csv(os.path.join(datapath, "TourneySeeds.csv"))
submit = pd.read_csv(os.path.join(datapath, submissionPath))

In [5]:
#teams_df.head()

In [6]:
#seeds_df.head()

In [7]:
df = seeds_df.merge(teams_df, left_on='Team', right_on='Team_Id')
keepcols = ['Season', 'Seed', 'Team_Id', 'Team_Name']
df = df.loc[df['Season'] == 2017, keepcols].reset_index(drop=True)

In [8]:
# create a binary tree and pipe things in the right place - create with seeds

# final four
#df[df['Team_Name'].isin(['Gonzaga', 'South Carolina', 'Oregon', 'North Carolina'])]

In [9]:
seedlist = list(reversed([
    'W01', 'W16', 'W08', 'W09',
    'W05', 'W12', 'W04', 'W13',
    'W06', 'W11', 'W03', 'W14',
    'W07', 'W10', 'W02', 'W15',
    'X01', 'X16', 'X08', 'X09',
    'X05', 'X12', 'X04', 'X13',
    'X06', 'X11', 'X03', 'X14',
    'X07', 'X10', 'X02', 'X15',
    'Y01', 'Y16', 'Y08', 'Y09',
    'Y05', 'Y12', 'Y04', 'Y13',
    'Y06', 'Y11', 'Y03', 'Y14',
    'Y07', 'Y10', 'Y02', 'Y15',
    'Z01', 'Z16', 'Z08', 'Z09',
    'Z05', 'Z12', 'Z04', 'Z13',
    'Z06', 'Z11', 'Z03', 'Z14',
    'Z07', 'Z10', 'Z02', 'Z15',
]))

In [10]:
#btree = build(range(135, 8, -1))
bracket_dict = dict(zip(range(135,8,-1),  list(range(63))+ seedlist))
bkt = build(bracket_dict.keys())

In [11]:
#bracket_dict

In [12]:
#bkt[135-58]

In [13]:
bracket_dict[1] = 'W11a'
bracket_dict[2] = 'W11b'
bracket_dict[3] = 'W16a'
bracket_dict[4] = 'W16b'
bracket_dict[5] = 'Y16a'
bracket_dict[6] = 'Y16b'
bracket_dict[7] = 'Z11a'
bracket_dict[8] = 'Z11b'


# need to double check these numbers maybe
bkt[135-18].left = extNode(1)
bkt[135-18].right = extNode(2)
bkt[135-10].left = extNode(3)
bkt[135-10].right = extNode(4)
bkt[135-42].left = extNode(5)
bkt[135-42].right = extNode(6)
bkt[135-66].left = extNode(7)
bkt[135-66].right = extNode(8)



# W11   18
# W16   10
# Y16   42
# Z11   66

In [14]:
with open("temp.txt", 'w') as f:
    f.write(str(bkt))

In [15]:
def get_team_id(seedMap):
    return (seedMap, df[df['Seed'] == bracket_dict[seedMap]]['Team_Id'].values[0])

In [16]:
levels = list(reversed(bkt.levels))
for level in levels:
    #print(level)
    for ix, node in enumerate(level[0:len(level)//2]):
        t1 = get_team_id(level[ix*2].value)
        t2 = get_team_id(level[ix*2+1].value)
        if t2[1] < t1[1]:
            temp = t1
            t1 = t2
            t2 = temp
        gid = '2017_'+str(t1[1])+'_'+str(t2[1])
        if submit[submit['Id']==gid]['Pred'].values[0] >= 0.5:
            level[ix*2].parent.value = t1[0]
        else: 
            level[ix*2].parent.value = t2[0]
        #print(level[ix*2].parent.value)

In [17]:
#need to fix this

In [18]:
from matchup_locs.ml2017 import matchup_locs
slotdata = []
for ix, key in enumerate([b for a in bkt.levels for b in a]):
#bracket_dict:
    xy = matchup_locs[135-ix]
    try:
        st = (
            str(bracket_dict[key.value]) 
            + ' ' 
            + str(df[df['Seed']==bracket_dict[key.value]]['Team_Name'].values[0])
        )
    except IndexError as e:
        st = str(bracket_dict[key.value])
    slotdata.append((xy, st))
#slotdata

In [19]:
# Create bracket image
# relevant:
# https://stackoverflow.com/questions/26649716/how-to-show-pil-image-in-ipython-notebook
emptyBracketPath = 'empty_brackets/2017.jpg'
img = Image.open(emptyBracketPath)
draw = ImageDraw.Draw(img)
# font = ImageFont.truetype(<font-file>, <font-size>)
# draw.text((x, y),"Sample Text",(r,g,b))
for slot in slotdata:
    draw.text(slot[0], str(slot[1]), (0, 0, 0))

dpi = 72
margin = 0.05  # (5% of the width/height of the figure...)
xpixels, ypixels = 940, 700

# Make a figure big enough to accomodate an axis of xpixels by ypixels
# as well as the ticklabels, etc...
figsize = (1 + margin) * ypixels / dpi, (1 + margin) * xpixels / dpi
fig = plt.figure(figsize=figsize, dpi=dpi)
# Make the axis the right size...
ax = fig.add_axes([margin, margin, 1 - 2*margin, 1 - 2*margin])

ax.imshow(np.asarray(img))
# plt.show() # for in notebook
img.save(outputFilePath)